In [ ]:
import pandas as pd
import re
import random
import torch
from tqdm.auto import tqdm

In [ ]:
# !pip install gdown
# import gdown
# file_id = ""
# url = f"https://drive.google.com/uc?id={file_id}"
# gdown.download(url, "ar_reviews_100k.tsv", quiet=False)


In [4]:

Arabic_Text = pd.read_csv("ar_reviews_100k.tsv", sep="\t")
Arabic_Text = Arabic_Text.rename(columns={"text": "Text", "label": "Label"})


In [ ]:
Arabic_Text.sample(3)

,Label,Text
98129,Negative,أتذكر أننى قرأت هذه الرواية فى زمن ليس ببعيد و...
47740,Mixed,قيمة الغرف . النظافة بوفيه الافطار. لا اعلم حق...
27113,Positive,جيد جدا. الموقع ممتاز. لاشيء يذكر


In [6]:
Arabic_Text['Label'].value_counts()

,count
Label,
Positive,33333
Mixed,33333
Negative,33333


In [7]:
Arabic_Text['Label'] =Arabic_Text['Label'].replace({"Mixed":"Neutral"})
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
Arabic_Text['Label'] =Arabic_Text['Label'].map(label_map)
Arabic_Text['Label'].value_counts()

,count
Label,
2,33333
1,33333
0,33333


In [8]:
Arabic_Text =Arabic_Text.dropna(subset=['Text'])
Arabic_Text = Arabic_Text.drop_duplicates(subset=['Text'])
print("========= check if there is null values in text ============")
print(Arabic_Text.isnull().sum())
print("========= check if there is duplicated values in text ============")
print(Arabic_Text.duplicated(subset=['Text']).sum())
print("========= Shape of Data ==============")
print(Arabic_Text.shape)

========= check if there is null values in text ============
Label    0
Text     0
dtype: int64
========= check if there is duplicated values in text ============
0
========= Shape of Data ==============
(99999, 2)


In [9]:
# ===============================
# Light Arabic Cleaning
# ===============================
def light_arabic_clean(text):
    text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", text)  # remove tashkeel
    text = re.sub(r"ـ+", "", text)                             # remove tatweel
    text = re.sub(r"\s+", " ", text).strip()                  # normalize spaces
    return text

In [10]:
from transformers import MarianMTModel, MarianTokenizer
import torch

# ===============================
# Load Translation Models
# ===============================
# Arabic → English
ar_en_tok = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
ar_en_mod = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en").eval()

# English → Arabic
en_ar_tok = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
en_ar_mod = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ar").eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
# ===============================
# Translation Functions
# ===============================
@torch.no_grad()
def translate(texts, tokenizer, model):
    tokens = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)# tokenizer : to make embedding for text
    outputs = model.generate(**tokens, max_length=128, num_beams=1)# generate translation after embedding -- out put is translation numbers(embeddings)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)#convert translation embedding into text to be readable

def back_translate(text):
    en = translate([text], ar_en_tok, ar_en_mod)[0]# from ar to en
    ar = translate([en], en_ar_tok, en_ar_mod)[0]# from en to ar
    return ar

In [12]:
# ===============================
# Optional: View Some Examples
# ===============================
sample_indices = random.sample(range(len(Arabic_Text)), 3)# select 3 random rows
for i in sample_indices:
    original = Arabic_Text.iloc[i]["Text"]# original text
    augmented = back_translate(original)# apply back translation
    print("Original:")
    print(original)
    print("After Back Translation:")
    print(augmented)
    print("-" * 60)

Original:
عدت من الفندق اليوم الفندق يقع في وسط المنطقة الصناعية يجب أن تستقل باص الفندق لمدة دقيقة للوصول إلى قرية تيوليب التي أجرت جزءا من شاطئها لفندق تيدا باص الفندق له مواعيد ذهاب ومثلهم للرجوع وبالحجز المسبق السائق كان محترما لكنه تخطى السرعات المقررة وسار عكس ا تجاه على الطريق الرئيسي وتحدث في التليفون أثناء القيادة الفندق يوجد به مساحات خضراء ويقع بين عدة مباني إدارية لشركات صينية وبالتالي يوجد فيو مفتوح المساحات الخارجية ضيقة وحول حمام السباحة ضيق وأيضا يستعمل كمكان لتناول الطعام في حالة امت ء المطعم الضيق الفندق أدوار نصف الغرف تطل على حمام السباحة والنصف ا خر خلفي وا سعار المعلنة هي للغرف الخلفية وأضف حوالي ج للفرد للغرف ا مامية على حمام السباحة يوجح مصعد خدمات الفندق تابع لشركة استثمار صيني مقرها مطل على الحمام وموظفوها منتشرون بالمكان ولهم مبنى كامل لسكنهم على حمام السباحة حمام السباحة لطيف وعنده غزل البنات ومرمى لكرة الماء ليلة السبت كان يوجد فرقة هابطة من أو د وبنتين يوجد رقعة شطرنج كبيرة وركن لعب أطفال وجيم وسبا الغرفة يوجد بها منشر لم بس البحر الحمام يوجد به مكان لتعلي

In [13]:
import random
from tqdm.auto import tqdm

# ===============================
# Back Translation
# ===============================
records = []

for _, row in tqdm(Arabic_Text.iterrows(), total=len(Arabic_Text)):# for loop to all rows
    text = row["Text"]
    label = row["Label"]

    # Originall Text after cleaning
    records.append({
        "text": light_arabic_clean(text),# apply clean to text
        "label": label
    })# records that have cleaned data

    # Back Translation on text that have less than  or equal 15 words
    if len(text.split()) <= 15 and random.random() < 0.5:  # 50% Random ---- example : when we get 30.000 sentence that have <= 15 words , i will select 15.000 from 30.000 to translate 
        try:
            aug_text = back_translate(text)  # Translate all Text
            records.append({
                "text": light_arabic_clean(aug_text),
                "label": label
            })
        except:
            pass

  0%|          | 0/99999 [00:00<?, ?it/s]

In [14]:
Arabic_Final = pd.DataFrame(records)

In [15]:
print(Arabic_Final.duplicated(subset=['text']).sum())
print(Arabic_Final['text'].isnull().sum())


481
0


In [16]:
Arabic_Final = Arabic_Final.dropna(subset=["text"])
Arabic_Final = Arabic_Final.drop_duplicates(subset=["text"])
print(Arabic_Final['text'].isnull().sum())
print(Arabic_Final.duplicated(subset=['text']).sum())

0
0


In [17]:
print(Arabic_Final.shape)
print(Arabic_Final.sample(5))

(115338, 2)
                                                     text  label
113821  رواية جيدة كفكرة ، صاحبها استطاع أن يوصل الحال...      0
103135  لم أتوقع أن يكون هذا سيئا الموقع فقط المصعد لا...      0
92878   للاسف اكتشفت انه غير نظيف لقيت في نفس التتبيله...      0
75362   مرضي. حرص الإدارة على رضاء و راحة العميل. اختل...      1
15797   الاقامة بالفندق كانت جيده . الفطور والخدمه. صغ...      2


In [18]:
print(Arabic_Final["label"].value_counts())


label
2    39075
1    38135
0    38128
Name: count, dtype: int64


In [19]:

Arabic_Final.to_csv(
    "arabic_sentence_level_back_translation.csv",
    index=False,
    encoding="utf-8-sig"
)
